In [ ]:
# %pip install s3fs seaborn xgboost pyarrow s3fs boto3==1.26.37 awswrangler --upgrade
%pip install xgboost boto3 awswrangler seaborn --upgrade

## Downloading the data

In [ ]:
import pandas as pd
import numpy as np
import boto3

# To handle import certain amount of rows for parquet files
import awswrangler as wr

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib

matplotlib.rcParams['figure.figsize'] = 10,10


#########################################
### SELECT INPUT
#########################################

Change dataset if desired

In [ ]:
%%time

# Using aws wrangler
train_s3_path = "s3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-2019/train/"
validation_s3_path = "s3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-2019/validation/"

train_df = wr.s3.read_parquet(path=train_s3_path, dataset=True)
validation_df = wr.s3.read_parquet(path=validation_s3_path, dataset=True)

validation_df


In [ ]:
train_df

In [ ]:
%%time

train_df.info()

### Identifying input and output columns

In [ ]:
%%time

train_df.corr()

In [ ]:
# input_col = ['vendor_id','pickup_at','dropoff_at', 'passenger_count','store_and_fwd_flag']
input_col = ['fare_amount','extra','tip_amount', 'tolls_amount','passenger_count', 'trip_distance']
target_col = 'total_amount'

In [ ]:
train_inputs = train_df[input_col].copy()
train_targets = train_df[target_col].copy()

In [ ]:
val_inputs = validation_df[input_col].copy()
val_targets = validation_df[target_col].copy()

In [ ]:
numeric_col = train_inputs.select_dtypes(include=np.number).columns.tolist()
cate_col = train_inputs.select_dtypes('object').columns.tolist()

In [ ]:
train_inputs[numeric_col]

In [ ]:
train_inputs[cate_col]

### Imputing missing values


Imputing is an technique in which the fill in the missing numeric values based on some category. Here if we want we'll use SimpleImputer form sklearn moduel

Reference:https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

In [ ]:
%%time

train_inputs[numeric_col].isna().sum()

In [ ]:
%%time

val_inputs[numeric_col].isna().sum()

It seems like there are no missing values in the train and validation datasets.

### Scaling numeric values

Now let's scale the numeric values to in range of (0,1) 

Reference:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
train_inputs[numeric_col].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
%%time

scaler.fit(train_inputs[numeric_col])

In [ ]:
%%time

train_inputs[numeric_col] = scaler.transform(train_inputs[numeric_col])
val_inputs[numeric_col]  = scaler.transform(val_inputs[numeric_col])

In [ ]:
train_inputs[numeric_col].describe()

### Encoding categorical columns

Now are going to encode categorical columns using one hot encoder into one hot numeric array
Reference:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder  = OneHotEncoder(sparse=False,handle_unknown='ignore')

In [ ]:
%%time

encoder.fit(train_inputs[cate_col])

In [ ]:
enc_col = encoder.get_feature_names(cate_col).tolist()
enc_col

In [ ]:
%%time

train_inputs[enc_col] = encoder.transform(train_inputs[cate_col])
val_inputs[enc_col] = encoder.transform(val_inputs[cate_col])

In [ ]:
train_inputs[enc_col]

In [ ]:
val_inputs

In [ ]:
%%time

x_train = train_inputs[numeric_col + enc_col]
x_val = val_inputs[numeric_col+enc_col]


## Model

## XGBoost

XGBoost is an powerful approach for solving regresion models.Since we are dealing regression problems, we are now going to use xgboost model
Reference:https://xgboost.readthedocs.io/en/latest/

In [ ]:
import xgboost

print(xgboost.__version__)

In [ ]:
from xgboost import XGBRegressor
model =  XGBRegressor(n_jobs=-1,random_state=42)

In [ ]:
%%time

model.fit(x_train,train_targets)

In [ ]:
%%time

model.predict(x_train)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(model.predict(x_train),train_targets,squared=False)

In [ ]:
mean_squared_error(model.predict(x_val),val_targets,squared=False)

## Hyperparameter tuning (Optional)

Now we have created a model, next we have to tune the model using the parameters so that we can reduce the loss
Refer:https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

### max_depth

In [ ]:
# def test_param(**params):
#   model = XGBRegressor(n_jobs=-1,random_state=42,**params)
#   model.fit(x_train,train_targets)
#   train_rmse = mean_squared_error(model.predict(x_train),train_targets,squared=False)
#   val_rmse = mean_squared_error(model.predict(x_val),val_targets,squared=False)
#   print(train_rmse," ",val_rmse)
#   return train_rmse,val_rmse

In [ ]:
# def test_param_and_plot(param_name, param_values):
#     train_errors, val_errors = [], [] 
#     for value in param_values:
#         params = {param_name: value}
#         train_rmse, val_rmse = test_param(**params)
#         train_errors.append(train_rmse)
#         val_errors.append(val_rmse)
#     plt.figure(figsize=(10,6))
#     plt.title('Overfitting curve: ' + param_name)
#     plt.plot(param_values, train_errors, 'b-o')
#     plt.plot(param_values, val_errors, 'r-o')
#     plt.xlabel(param_name)
#     plt.ylabel('RMSE')
#     plt.legend(['Training', 'Validation'])

In [ ]:
# test_param(max_depth=2)

In [ ]:
# test_param(max_depth=5)

In [ ]:
# test_param(max_depth=10)

In [ ]:
# test_param_and_plot('max_depth',[x for x in range(0,60,10)])

### n_estimators

In [ ]:
# test_param(n_estimators=10)

In [ ]:
# test_param(n_estimators=20)

In [ ]:
# test_param(n_estimators=30)

In [ ]:
# test_param(n_estimators=40)

In [ ]:
# test_param(n_estimators=50)

In [ ]:
# test_param(n_estimators=50,max_depth=30)

In [ ]:
# test_param(n_estimators=60,max_depth=30)

In [ ]:
# test_param_and_plot('n_estimators',[x for x in range(100,1100,100)])

### learning_rate

In [ ]:
# test_param(learning_rate=0.01)

In [ ]:
# test_param(learning_rate=0.1)

In [ ]:
# test_param(learning_rate=0.3)

In [ ]:
# test_param(learning_rate=0.5)

In [ ]:
# test_param(learning_rate=0.9)

In [ ]:
# test_param(learning_rate=1)

In [ ]:
# test_param(learning_rate=1,max_depth=60,n_estimators=60)

In [ ]:
# test_param(learning_rate=1,max_depth=60,n_estimators=100)

### booster

In [ ]:
# test_param(booster='gblinear')

In [ ]:
# test_param(booster='gblinear', max_depth=60, n_estimators=100, learning_rate=1)

In [ ]:
# test_param(booster='gbtree',max_depth=60,n_estimators=100,learning_rate=1)

## Putting it together

In [ ]:
# model = XGBRegressor(n_jobs=-1,random_state=42,max_depth=60,n_estimaotrs=100,learning_rate=0.9,booster='gbtree')

In [ ]:
# model.fit(x_train,train_targets)

In [ ]:
# mean_squared_error(model.predict(x_train),train_targets,squared=False)

In [ ]:
# mean_squared_error(model.predict(x_val),val_targets,squared=False)

In [ ]:
# mean_squared_error(model.predict(x_test),test_targets,squared=False)